In [1]:
import re
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mariagiustiniano/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!mv /Users/mariagiustiniano/Downloads/kaggle.json  ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download aashita/nyt-comments

In [ ]:
!unzip 'nyt-comments' -d /Users/mariagiustiniano/com_1

In [ ]:
folder = '/Users/mariagiustiniano/com_1'
filelist = [[file.split('.')[0],os.path.join(folder,file)] for file in os.listdir('/Users/mariagiustiniano/com_1')]

db = {}
db_a = {}

for file in filelist:
    if file[0].split('.')[0].startswith('C'):
        db[file[0].split('.')[0]] = pd.read_csv(file[1], low_memory=False)
    else:
        db_a[file[0].split('.')[0]] = pd.read_csv(file[1], low_memory=False)

In [4]:
keys= list(db.keys())
keys_a=list(db_a.keys())

In [ ]:
sia= SentimentIntensityAnalyzer()
for key in keys:
    db.get(key)['editorsSelection']= db.get(key)['editorsSelection'].replace([True,False],[1,0])
    db.get(key)['replyCount']= db.get(key)['replyCount'].astype(float).apply(abs)
    db.get(key)['recommendations']= db.get(key)['recommendations'].astype(float).apply(abs)
    db.get(key)['compound']= db.get(key)['commentBody'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [5]:
def norm(x):
    s= (x-min(x))/(max(x)-min(x))
    return s

In [6]:
def weight(row):
    """Return a weight for each commentID"""
    x= row['editorsSelection']
    y= row['normRecommendations']
    z= row['normReplyCount']
    return x+y+z

In [7]:
for key in keys:
    db[key]['normReplyCount']= norm(db[key]['replyCount'])
    db[key]['normRecommendations']= norm(db[key]['recommendations'])
    db[key]['weight']= db[key][['editorsSelection','normRecommendations','normReplyCount']].sum(axis=1)

In [8]:
Scores_comm= {}
for key in keys:
    for i in list(range(db[key].shape[0])): 
        Scores_comm[db[key].commentID[i]] = db[key].weight[i]

In [9]:
#Update comments
Rep_score={}
for key in keys:
    Rep_score[key]= pd.DataFrame({'score_std': db[key].groupby(['inReplyTo'])['weight'].std().fillna(0)}).iloc[1:].reset_index()
    Rep_score[key]['sd']= Rep_score[key].inReplyTo.map(db[key].groupby(['inReplyTo'])['compound'].std().fillna(0))
    Rep_score[key]['score']= Rep_score[key][['score_std','sd']].sum(axis=1)
    for i in Rep_score[key].inReplyTo:
        Scores_comm[i]= Scores_comm[i]+ Rep_score[key].score[Rep_score[key].inReplyTo == i].values[0]

In [10]:
articles={} 
Score_art={}
for key in keys:
    articles[key]= pd.DataFrame({'list_comments' : db[key][db[key].commentType=='comment'].groupby('articleID')['commentID'].apply(list)}).reset_index()
    articles[key]['sd']= articles[key].articleID.map(db[key][db[key].commentType=='comment'].groupby(['articleID'])['compound'].std().fillna(0))
    for j in list(range(articles[key].shape[0])):
        Score_art[articles[key].articleID[j]]= np.std([Scores_comm[comment] for comment in articles[key].list_comments[j]])+articles[key].sd[j]

In [11]:
topic={}
mean={}
for key in keys:
    topic[key]= pd.DataFrame({'art_ID': db[key].groupby('sectionName')['articleID'].unique().apply(list)}).reset_index()
    mean[key]=[]
    for i in list(range(topic[key].shape[0])):
        mean[key].append(np.mean([Score_art[k] for k in topic[key].art_ID[i]]))
    topic[key]['rate_succ']= mean[key]
    topic[key].sort_values(by='rate_succ',inplace=True, ascending=False)